In [1]:
import os
import pickle
import time
from collections import namedtuple
from datetime import datetime, timedelta

import pandas as pd
import requests
import tqdm
import matplotlib.pyplot as plt

import timeit
import functools

%matplotlib inline

In [2]:
API_KEY = '340abe7ef770de43932468b8cd663dba'
BASE_URL = 'https://api.darksky.net/forecast/{}/{},{}'

In [3]:
features = [
    'date',
    'temperatureMean',
    'dewPoint',
    'pressure',
    'humidity',
    'temperatureMax',
    'temperatureMin',
    'precipProbability',
]
DailySummary = namedtuple('DailySummary', features)



In [4]:
# timer decorator
def timer(func):
    """Print function runtime"""

    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start_time = timeit.default_timer()
        result = func(*args, **kwargs)
        elapsed = timeit.default_timer() - start_time
        print('Function {name!r} took {time:.4f} secs.'.format(name=func.__name__, time=elapsed))
        return result

    return wrapper_timer

In [18]:
@timer
def extract_weather_data(url, api_key, latlng, target_date, days):
    records = []
    for _ in tqdm.tqdm(range(days)):
        request = BASE_URL.format(
             API_KEY, latlng, target_date.strftime('%Y-%m-%dT%H:%M:%S')
        )
        response = requests.get(request)
        if response.status_code == 200:

            def get_mean_temp():
                """Return average temperature across a 24 hour period."""
                total_temp = 0
                for i in range(len(hdata)):
                    try:
                        total_temp += hdata[i]['temperature']
                    except KeyError:
                        total_temp += hdata[i-1]['temperature']
                meanTemp = total_temp / 24
                return meanTemp

            data = response.json()['daily']['data'][0]
            hdata = response.json()['hourly']['data']
            try:
                records.append(
                    DailySummary(
                        date=target_date,
                        temperatureMean=get_mean_temp(),
                        dewPoint=data['dewPoint'],
                        pressure=data['pressure'],
                        humidity=data['humidity'],
                        temperatureMax=data['temperatureMax'],
                        temperatureMin=data['temperatureMin'],
                        precipProbability=data['precipProbability'],
                    )
                )
            except KeyError:
                records.append(
                    DailySummary(
                        date=target_date,
                        temperatureMean=get_mean_temp(),
                        dewPoint=data['dewPoint'],
                        pressure=data['pressure'],
                        humidity=data['humidity'],
                        temperatureMax=data['temperatureMax'],
                        temperatureMin=data['temperatureMin'],
                        precipProbability=0,
                    )
                )
        # time.sleep(6)
        target_date += timedelta(days=1)
    return records



In [19]:
# Do not run this cell when collecting data on day 2
def get_target_date():
    """Return target date 1000 days prior to current date."""
    current_date = datetime.now()
    target_date = current_date - timedelta(days=5)
    return target_date

target_date = get_target_date()



In [20]:
latlng = '26.5833,87.9167'
records = extract_weather_data(BASE_URL, API_KEY, latlng, target_date, 5)

  0%|          | 0/5 [00:01<?, ?it/s]

Function 'extract_weather_data' took 1.5905 secs.


In [21]:
len(records)

TypeError: object of type 'Response' has no len()

In [9]:
records

[DailySummary(date=datetime.datetime(2020, 8, 23, 20, 34, 19, 763324), temperatureMean=81.42, dewPoint=77.6, pressure=1004.7, humidity=0.88, temperatureMax=88.59, temperatureMin=74.48, precipProbability=0.91),
 DailySummary(date=datetime.datetime(2020, 8, 24, 20, 34, 19, 763324), temperatureMean=83.13541666666667, dewPoint=79.61, pressure=1003.5, humidity=0.89, temperatureMax=90.15, temperatureMin=75.77, precipProbability=0.54),
 DailySummary(date=datetime.datetime(2020, 8, 25, 20, 34, 19, 763324), temperatureMean=86.21750000000002, dewPoint=79.87, pressure=1004.4, humidity=0.82, temperatureMax=95.03, temperatureMin=77.67, precipProbability=0.17),
 DailySummary(date=datetime.datetime(2020, 8, 26, 20, 34, 19, 763324), temperatureMean=83.98291666666665, dewPoint=77.2, pressure=1005.7, humidity=0.8, temperatureMax=90.8, temperatureMin=76.28, precipProbability=0.36),
 DailySummary(date=datetime.datetime(2020, 8, 27, 20, 34, 19, 763324), temperatureMean=82.93916666666668, dewPoint=75.55, pr

In [10]:
# save records list
# with open('records_pt1.pkl', 'wb') as f:
#     pickle.dump(records, f)



In [11]:
df = pd.DataFrame(records, columns=features).set_index('date')

In [14]:
current_time = datetime.now()
current_time.strftime("%Y-%m-%d")

'2020-08-28'

In [17]:
df.to_csv('darksky/data-' + current_time.strftime("%Y-%m-%d") + '.csv')